In [ ]:
import seaborn as sns
sns.set()
import dill

In [ ]:
from static_grader import grader

# NLP: Analyzing Review Text


Unstructured data makes up the vast majority of data.  This is a basic intro to handling unstructured data.  Our objective is to be able to extract the sentiment (positive or negative) from review text.  We will do this from Yelp review data.

The first three questions task you to build models, of increasing complexity, to predict the rating of a review from its text.  These models will be assessed based on the root mean squared error of the number of stars predicted.  There is a reference solution (which should not be too hard to beat) that defines the score of 1.

The final question asks only for the result of a calculation, and your results will be compared directly to those of a reference solution.


## A note on scoring

It **is** possible to score >1 on some of these questions. This indicates that you've beaten our reference model - we compare our model's score on a test set to your score on a test set. See how high you can go!


## Download and parse the data


To start, let's download the data set from Amazon S3:

In [ ]:
!aws s3 sync s3://dataincubator-course/mldata/ . --exclude '*' --include 'yelp_train_academic_dataset_review_reduced.json.gz'

The training data are a series of JSON objects, in a Gzipped file. Python supports Gzipped files natively: [`gzip.open`](https://docs.python.org/3/library/gzip.html) has the same interface as `open`, but handles `.gz` files automatically.

The built-in json package has a `loads()` function that converts a JSON string into a Python dictionary.  We could call that once for each row of the file. [`ujson`](http://docs.micropython.org/en/latest/library/ujson.html) has the same interface as the built-in `json` library, but is *substantially* faster (at the cost of non-robust handling of malformed json).  We will use that inside a list comprehension to get a list of dictionaries:

In [ ]:
import gzip
import ujson as json

with gzip.open('yelp_train_academic_dataset_review_reduced.json.gz') as f:
    data = [json.loads(line) for line in f]

Scikit Learn will want the labels in a separate data structure, so let's pull those out now.

In [ ]:
stars = [row['stars'] for row in data]
text = [row['text'] for row in data]

In [ ]:
import pandas as pd
df = pd.DataFrame(data)
df.head()

### Notes:

1. [Pandas](http://pandas.pydata.org/) is able to read JSON text directly.  Use the `read_json()` function with the `lines=True` keyword argument.  While the rest of this notebook will assume you are using a list of dictionaries, you can complete it with dataframes, if you so desire. Some of the example code will need to be modified in this case.

2. There are obvious mistakes in the data.  There is no need to try to correct them.


## Building models


For the first two questions, you will need to build and train an estimator to predict the star rating from the text of a review.  We recommend building a pipeline out of transformers and estimators provided by Scikit Learn.  You can decide whether these pipelines should take full review objects or just their text as input to the `fit()` and `predict()` methods, but it does pay to be consistent.

You may find it useful to serialize the trained models to disk.  This will allow to reload it after restarting the Jupyter notebook, without needing to retrain it.  We recommend using the [`dill` library](https://pypi.python.org/pypi/dill) for this (although the [`joblib` library](http://scikit-learn.org/stable/modules/model_persistence.html) also works).  Use
```python
with open('estimator.dill', 'wb') as f:
    dill.dump(estimator, f)
```
to serialize the object `estimator` to the file `estimator.dill`.  If you have trouble with this, try setting the `recurse=True` keyword arguments in the call of `dill.dump()`.  The estimator can be deserialized by calling
```python
with open('estimator.dill', 'rb') as f:
    estimator = dill.load(f)
```

You may notice that your serialized models are very large and take a long time to load.  Some hints to reduce their size:

- If you are using `GridSearchCV` to find the optimal values of hyperparameters (and you should be), the resultant object will contain many copies of the estimator that aren't needed any more.  Instead of serializing the whole `GridSearchCV`, serialize just the estimator with the correct hyperparameters.  This can be accessed through the `.best_estimator_` attribute of the `GridSearchCV` object.  Alternatively, the `.best_params_` attribute gives the best values of the hyperparameters.

- The `CountVectorizer` keeps track of all words that were excluded from vectorization in its `.stop_words_` attribute.  This can be interesting to examine, but isn't needed for predictions.  Set this attribute to the empty list before serializing it to save disk space.

# Questions


Each of the "model" questions asks you to create a function that models the number of stars given in a review from the review text.  It will be passed a list of dictionaries.  Each of these will have the same format as the JSON objects you've just read in.  This function should return a list of numbers of the same length, giving the predicted star ratings.

This function is passed to the `score()` function, which will receive input from the grader, run your function with that input, report the results back to the grader, and print out the score the grader returned.  Depending on how you constructed your estimator, you may be able to pass the predict method directly to the `score()` function.  If not, you will need to write a small wrapper function to mediate the data types.


## bag_of_words_model

Build a linear model predicting the star rating based on the count of the words in each document (bag-of-words model).  Use a [`CountVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer) to produce a feature matrix giving the counts of each word in each review.  Feed the feature matrix into a linear model, such as `Ridge` or `SGDRegressor`, to predict the number of stars from each review.

**Hints**:
1. Don't forget to use tokenization!  This is important for good performance but it is also the most expensive step.  Try vectorizing as a first initial step and then running grid-search and cross-validation only on of this pre-processed data.  `CountVectorizer` has to memorize the mapping between words and the index to which it is assigned.  This is linear in the size of the vocabulary.

```python
from sklearn.feature_extraction.text import CountVectorizer

text = [row['text'] for row in data]
X = CountVectorizer().fit_transform(text)

# Now, this can be run with many different parameters
# without needing to retrain the vectorizer:
model.fit(X, stars, hyperparameter=something)
```

2. Try choosing different values for `min_df` (minimum document frequency cutoff) and `max_df` in `CountVectorizer`.  Setting `min_df` to zero admits rare words which might only appear once in the entire corpus.  This is both prone to overfitting and makes your data unmanageably large.  Don't forget to use cross-validation to select the right value.  
3. Try using [`LinearRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) or [`RidgeCV`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html#sklearn.linear_model.RidgeCV).  If the memory footprint is too big, try switching to [Stochastic Gradient Descent](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html#sklearn.linear_model.SGDRegressor) You might find that even ordinary linear regression fails due to the data size.  Don't forget to use [`GridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) to determine the regularization parameter!  How do the regularization parameter `alpha` and the values of `min_df` and `max_df` from `CountVectorizer` change the answer?

4. You will likely pick up several hyperparameters between the tokenization step and the regularization of the estimator.  While it is more strictly correct to do a grid search over all of them at once, this can take a long time. Quite often, doing a grid search over a single hyperparameter at a time can produce similar results.  Alternatively, the grid search may be done over a smaller subset of the data, as long as it is representative of the whole.

5. Finally, assemble a pipeline that will transform the data from records all the way to predictions.  This will allow you to submit its predict method to the grader for scoring.

In [ ]:
import spacy
nlp = spacy.load('en')
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer, CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

In [ ]:
model = GridSearchCV(Ridge(), param_grid={'alpha': [100,500]}, scoring='neg_mean_squared_error',cv=5)

In [ ]:
STOP_WORDS = spacy.lang.en.stop_words.STOP_WORDS
def tokenize_lemma(text):
    return [w.lemma_.lower() for w in nlp(text)]

stop_words_lemma = set(tokenize_lemma(' '.join(STOP_WORDS)))

In [ ]:
bag_of_words_vectorizer = HashingVectorizer(n_features = 1000, token_pattern=None, 
                                            stop_words=stop_words_lemma, tokenizer = tokenize_lemma)

In [ ]:
from sklearn import base
class ColumnTransformer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, col_names):
        self.col_names = ['text']
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        self.X = X
        list_of_texts =[row['text'] for row in self.X]
        return list_of_texts

In [ ]:
from sklearn.pipeline import Pipeline
bag_of_words_est = Pipeline([
                            ('Transformer', ColumnTransformer(['text'])),
                            ('Vectorizer', bag_of_words_vectorizer),
                            ('Classifier' , Ridge(alpha=430))                           
])

In [ ]:
bag_of_words_est.fit(data, stars)

In [ ]:
dill.dump(bag_of_words_est, open("NLPQ1","wb")) 

In [ ]:
grader.score('nlp__bag_of_words_model', bag_of_words_est.predict)

## bigram_model


In a bigram model, we'll consider both single words and pairs of consecutive words that appear.  This is going to be a much higher-dimensional problem (large $p$) so you should be careful about overfitting.  You could also use normalization (the `TfidfVectorizer` or a word count vectorizer combined with `TfidfTransformer`)  

Sometimes, reducing the dimension can be useful.  If you're using the `CountVectorizer` or `TfidfVectorizer`, you can change the `max_features` hyperparameter to reduce the size of the resulting vocabulary.  This isn't possible with the `HashingVectorizer`.  (Why?)  

Playing with the number of features, using the `HashingVectorizer`, incorporating `min_df` and `max_df` limits, and handling stopwords in some way are all methods of addressing the issue of high dimensionality. If you are using `CountVectorizer`, it is possible to run it with a fixed vocabulary (based on a training run, for instance). Check the documentation.

**A side note on multi-stage model evaluation:** When your model consists of a pipeline with several stages, it can be worthwhile to evaluate which parts of the pipeline have the greatest impact on the overall accuracy (or other metric) of the model. This allows you to focus your efforts on improving the important algorithms, and leaving the rest "good enough".

One way to accomplish this is through ceiling analysis, which can be useful when you have a training set with ground truth values at each stage. Let's say you're training a model to extract image captions from websites and return a list of names that were in the caption. Your overall accuracy at some point reaches 70%. You can try manually giving the model what you know are the correct image captions from the training set, and see how the accuracy improves (maybe up to 75%). Alternatively, giving the model the perfect name parsing for each caption increases accuracy to 90%. This indicates that the name parsing is a much more promising target for further work, and the caption extraction is a relatively smaller factor in the overall performance.

If you don't know the right answers at different stages of the pipeline, you can still evaluate how important different parts of the model are to its performance by changing or removing certain steps while keeping everything else constant. You might try this kind of analysis to determine how important adding stopwords and stemming to your NLP model actually is, and how that importance changes with parameters like the number of features.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
def tokenize_lemma(text):
    return [word.lemma_.lower() for word in nlp(text)]
stop_words_lemma = set(tokenize_lemma(' '.join(STOP_WORDS)))
tfidf = TfidfVectorizer(max_features=10000,
                               ngram_range=(1,2),
                               stop_words=stop_words_lemma,
                               tokenizer=tokenize_lemma,
                               token_pattern=None,
                               )

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import SGDRegressor
from sklearn.feature_extraction.text import HashingVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline

model = GridSearchCV(Ridge(), param_grid={'alpha':np.logspace(-10,5,20)},cv=5)

bigram_pipeline = Pipeline([
                            ('Transformer', ColumnTransformer(['text'])), 
                            ('vectorize', tfidf), 
                            ('estimator',model),
                        ])
bigram_pipeline.fit(data,stars)

In [ ]:
dill.dump(bigram_pipeline, open("NLPQ2","wb")) 

In [ ]:
grader.score('nlp__bigram_model', bigram_pipeline.predict) 

## word_polarity

Let's consider a different approach and try to derive some insight from our analysis.  

We want to determine the most "polarizing words" in the corpus of reviews.  In other words, we want to identify words that strongly signal a review is either positive or negative.  For example, we understand that a word like "terrible" will most likely appear in negative rather than positive reviews.  

During training, the [naive Bayes model](https://scikit-learn.org/stable/modules/naive_bayes.html#) calculates probabilities such as $Pr(\textrm{terrible}\ |\ \textrm{negative}),$ the probability that the word "terrible" appears in the review text, given that the review is negative.  Using these probabilities, we can define a **polarity score** for each word $w$,

$$\textrm{polarity}(w) = \log\left(\frac{Pr(w\ |\ \textrm{positive})}{Pr(w\ |\ \textrm{negative})}\right).$$

Polarity analysis is an example where a simpler model (naive Bayes) offers more explicability than more complicated models.  Aside from this, naive Bayes models are easy to train, the training process is parallelizable, and these models lend themselves well to online learning.  Given enough training data, naive Bayes models have performed well in NLP applications such as spam filtering.  

For this problem, you are asked to determine the top 25 most positive polar words and the 25 most negative polar words.  For this analysis, you should:

1.  **Filter** the collection of reviews you were using above to **only keep** the one-star and five-star reviews. Since these are the "most polar" reviews, it should give us the most polarizing words.   
1.  Use the naive Bayes model, `MultinomialNB`.  
1.  Use TF-IDF weighting.
1.  Remove stop words.
1.  As mentioned, generate a (Python) list with most positive (25 words) and most negative (25 words) polar words.  

A naive Bayes model (after training) stores the log of the probabilities in an attribute of the model.  It is a `numpy` array of shape (number of classes, number of features).  You will need the mapping between feature indices to words to find the most polarizing words.  

In [ ]:
positive_stars = [row['stars'] for row in data if row['stars']==5]

In [ ]:
negative_stars = [row['stars'] for row in data if row['stars']==1]

In [ ]:
positive_text = [row['text'] for row in data if row['stars']==5]

In [ ]:
negative_text = [row['text'] for row in data if row['stars']==1]

In [ ]:
new_stars = [row['stars'] for row in data if row['stars']==1 or row['stars']==5]

In [ ]:
polar_data = [row for row in data if row['stars']==1 or row['stars']==5]

In [ ]:
label  = ['positive']*len(positive_text) + ['negative']*len(negative_text)

In [ ]:
assert len(polar_data) == 116576

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
ng_stem_tfidf = TfidfVectorizer(
                               stop_words=STOP_WORDS
                               )

In [ ]:
fitted_vec = ng_stem_tfidf.fit(positive_text+negative_text)

In [ ]:
names = fitted_vec.get_feature_names()

In [ ]:
tran_vec = fitted_vec.transform(positive_text+negative_text)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf =MultinomialNB()
clf.fit(tran_vec,label)

In [ ]:
prob = clf.feature_log_prob_

In [ ]:
pol = prob[0]- prob[1]

In [ ]:
merged_list = [(names[i], pol[i]) for i in range(0, len(names))]

In [ ]:
sorted_list = sorted(merged_list, key=lambda x: x[1])
features = [x for (x,y) in sorted_list]

In [ ]:
poss = features[:25]
neg = features[-25:]
polar_words =poss+neg

In [ ]:
# polar_words = ['perfection'] * 50

In [ ]:
grader.score('nlp__word_polarity', polar_words)

## food_bigrams

Look over all reviews of restaurants.  You can determine which businesses are restaurants by looking in the `yelp_train_academic_dataset_business.json.gz` file from the ml project or downloaded below.

In [ ]:
!aws s3 sync s3://dataincubator-course/mldata/ . --exclude '*' --include 'yelp_train_academic_dataset_business.json.gz'

In [ ]:
with gzip.open('yelp_train_academic_dataset_business.json.gz') as f:
    business_data = [json.loads(line) for line in f]

Each row of this file corresponds to a single business.  The category key gives a list of categories for each; take all where "Restaurants" appears.

In [ ]:
restaurant_ids= [row['business_id'] for row in business_data if 'Restaurants' in row['categories']]
restaurant_data =  [row for row in business_data if 'Restaurants' in row['categories']]

In [ ]:
assert len(restaurant_ids) == 12876

In [ ]:
df_Restaurants = pd.DataFrame(restaurant_data)
df_review = pd.DataFrame(data)

The "business_id" here is the same as in the review data.  Use this to extract the review text for all reviews of restaurants.

In [ ]:
restaurant_reviews = pd.merge(df_review,df_Restaurants, how = 'inner', on = 'business_id')

In [ ]:
text_new = restaurant_reviews['text'].values.tolist()
Y = restaurant_reviews['stars_x'].values.tolist()

In [ ]:
assert len(restaurant_reviews) == 143361

We want to find collocations --- that is, bigrams that are "special" and appear more often than you'd expect from chance. We can think of the corpus as defining an empirical distribution over all *n*-grams.  We can find word pairs that are unlikely to occur consecutively based on the underlying probability of their words. Mathematically, if $p(w)$ be the probability of a word $w$ and $p(w_1 w_2)$ is the probability of the bigram $w_1 w_2$, then we want to look at word pairs $w_1 w_2$ where the statistic

  $$ \frac{p(w_1 w_2)}{p(w_1) p(w_2)} $$

is high.  Return the top 100 (mostly food) bigrams with this statistic with the 'right' prior factor (see below).

Estimating the probabilities is simply a matter of counting, and there are number of approaches that will work.  One is to use one of the tokenizers to count up how many times each word and each bigram appears in each review, and then sum those up over all reviews.  You might want to know that the `CountVectorizer` has a `.get_feature_names()` method which gives the string associated with each column.  (Question for thought: Why doesn't the `HashingVectorizer` have a similar method?)

*Questions:* This statistic is a ratio and problematic when the denominator is small.  We can fix this by applying Bayesian smoothing to $p(w)$ (i.e. mixing the empirical distribution with the uniform distribution over the vocabulary).

1. How does changing this smoothing parameter affect the word pairs you get qualitatively?

2. We can interpret the smoothing parameter as adding a constant number of occurrences of each word to our distribution.  Does this help you determine set a reasonable value for this 'prior factor'?

3. For fun: also check out [Amazon's Statistically Improbable Phrases](http://en.wikipedia.org/wiki/Statistically_Improbable_Phrases).

*Implementation note:*
As you adjust the size of the Bayesian smoothing parameter, you will notice first nonsense phrases being removed and then legitimate bigrams being removed, leaving you with only generic bigrams.  The goal is to find a value of the smoothing parameter between these two transitions.

The reference solution is not an aggressive filterer: it errors in favor of leaving apparently nonsensical words. On further consideration, many of these are actually somewhat meaningful. The smoothing parameter chosen in the reference solution is equivalent to giving each word 30 previous appearances prior to considering this data.  This was chosen by generating a list of bigrams for a range of smoothing parameters and seeing how many of the bigrams were shared between neighboring values.  When the shared fraction reached 95%, we judged the solution to have converged.  Note that `min_df` should not be set too high, where it could exclude these borderline words.

In [ ]:
#Create Unigram Model convert a collection of text documents to a matrix of token counts
unigram_vect = CountVectorizer(ngram_range=(1,1),stop_words= stop_words_lemma,min_df=0.0001,max_features = 100000,strip_accents = "ascii")
X_train_unigram = unigram_vect.fit_transform(text_new)

# Create Bigram Model
bigram_vect = CountVectorizer(ngram_range=(2,2),stop_words= stop_words_lemma, min_df=0.0001,max_features = 100000,strip_accents = "ascii")
X_train_bigram = bigram_vect.fit_transform(text_new)

In [ ]:
bicount_list = X_train_bigram.sum(axis=0).tolist()[0]
total_count_biword = X_train_bigram.sum()
total_count_uniword = X_train_unigram.sum()
unicount_list = X_train_unigram.sum(axis = 0).tolist()[0]
uniword_list = sorted(unigram_vect.vocabulary_, key=unigram_vect.vocabulary_.get)
biword_list = sorted(bigram_vect.vocabulary_, key=bigram_vect.vocabulary_.get)

In [ ]:
# build a dictionary for biwords and uniword
l=[]
biword_dict = dict(zip(biword_list,bicount_list))
uniword_dict = dict(zip(uniword_list,unicount_list))
for word_pair, counts in biword_dict.items():
    w1 = word_pair.split(' ')[0]
    w2 = word_pair.split(' ')[1]
    item = (w1,w2 ,uniword_dict[w1],uniword_dict[w2],counts)
    l.append(item)

In [ ]:
df_ratio = pd.DataFrame(l)
df_ratio.columns = ['w1','w2','count1','count2','counts']
df_ratio['ratio'] = (df_ratio.counts)/((df_ratio.count1)*(df_ratio.count2))

In [ ]:
# Create a List of Biwords
result = df_ratio.sort_values(['ratio'], ascending=[0])
result_ = result[['w1','w2']].values.tolist()
final_l = []
for item in result_:
    new_item = item[0]+" "+item[1]
    final_l.append(new_item)

In [ ]:
import re
# remove digits in the string 
RE_D = re.compile('\d|\_')
final_l_str = []
for item in final_l:
    if RE_D.search(item):
        continue
    else: 
        final_l_str.append(item)

In [ ]:
top100=final_l_str[:100]

In [ ]:

# top100 = ['haricot vert'] * 100

In [ ]:
grader.score('nlp__food_bigrams', top100)

*Copyright &copy; 2021 Pragmatic Institute. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*